In [2]:
import utils
import sagemaker
import json
import mlflow

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [22]:
# Detecting Execution Role
sagemaker.get_execution_role()

'arn:aws:iam::654654589924:role/service-role/SageMaker-MLOpsEngineer'

# Loading and saving the MLFlow model locally

In [4]:
# Set tracking server
tracking_server_arn = 'arn:aws:sagemaker:us-east-1:654654589924:mlflow-tracking-server/mlops-utec-mlflow-server'
mlflow.set_tracking_uri(tracking_server_arn)

In [5]:
# Load model
model_name = "credit-card-fraud-detection"
model_version = "latest"
model_uri = f"models:/{model_name}/{model_version}"
model = mlflow.xgboost.load_model(model_uri)

In [6]:
# Save model locally
mlflow.xgboost.save_model(model,"model")

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:54:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1744329020674/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


# Checking current images

In [7]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


# Creating ECR repository

In [8]:
user = utils.get_username()
repository_name = f"utec-mlops/{user}/online-prediction/api/rest/fastapi"
repository_name

'utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi'

In [9]:
!aws ecr create-repository --repository-name $repository_name

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:654654589924:repository/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi",
        "registryId": "654654589924",
        "repositoryName": "utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi",
        "repositoryUri": "654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi",
        "createdAt": "2025-06-12T20:55:09.019000+00:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


# Build image

In [10]:
!docker build --network sagemaker --no-cache -t $repository_name .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.

Sending build context to Docker daemon  161.3kB
Step 1/8 : FROM python:3.8
3.8: Pulling from library/python

2bf39133: Pulling fs layer 
ff7f31e9: Pulling fs layer 
f2aee8e9: Pulling fs layer 
2fe8adba: Pulling fs layer 
73e4e6c7: Pulling fs layer 
f13b5f0f: Pulling fs layer 
Digest: sha256:d411270700143fa2683cc8264d9fa5d3279fd3b6afff62ae81ea2f9d070e390c
Status: Downloaded newer image for python:3.8
 ---> 3ea6eaad4f17
Step 2/8 : WORKDIR /app
 ---> Running in 6127d8c657a7
 ---> Removed intermediate container 6127d8c657a7
 ---> 8fc38c021e7d
Step 3/8 : COPY model /app/model
 ---> 1e0354df2dc4
Step 4/8 : COPY app.py requirements.txt /app/
 ---> ff9a117b0e1a
Step 5/8 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Running in e02bd216d4e1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Checking current images

In [11]:
!docker images

REPOSITORY                                                   TAG       IMAGE ID       CREATED          SIZE
utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi   latest    cc6e2b6e5849   55 seconds ago   2.43GB


# Login to ECR

In [12]:
ecr_url = "654654589924.dkr.ecr.us-east-1.amazonaws.com"

In [13]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin $ecr_url

WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores

Login Succeeded


# Tag image

In [14]:
tag_name = f"{repository_name}:latest"
full_tag_name = f"{ecr_url}/{tag_name}"
print(tag_name)
print(full_tag_name)

utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi:latest
654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi:latest


In [15]:
!docker tag $tag_name $full_tag_name

# Checking current images

In [16]:
!docker images

REPOSITORY                                                                                                TAG       IMAGE ID       CREATED              SIZE
654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi   latest    cc6e2b6e5849   About a minute ago   2.43GB
utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi                                                latest    cc6e2b6e5849   About a minute ago   2.43GB


# Push image to ECR

In [17]:
!docker push $full_tag_name

The push refers to repository [654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/ricardo-puma/online-prediction/api/rest/fastapi]

fa672ce1: Preparing 
28026ad9: Preparing 
a2573129: Preparing 
d92e2b01: Preparing 
710ca3c7: Preparing 
e4d52b5a: Preparing 
8afd69b3: Preparing 
433c3a29: Preparing 
c7a486d9: Preparing 
a6961052: Preparing 
latest: digest: sha256:e51714f4824442d0b1056b4d2775b45010600378cec86b0290a06f87ebe04eac size: 2630


# Create App Runner Service

In [23]:
service_name = f"utec-mlops-{user}-fastapi"
app_runner_role = "arn:aws:iam::654654589924:role/service-role/AppRunnerECRAccessRole"

In [19]:
conf = {
    "ServiceName": service_name,
    "SourceConfiguration": {
        "AuthenticationConfiguration": {
            "AccessRoleArn": app_runner_role
        },
        "ImageRepository": {
            "ImageIdentifier": full_tag_name,
            "ImageConfiguration": {
                "Port": "8000"
            },
            "ImageRepositoryType": "ECR"
        }
    },
    "InstanceConfiguration": {
        "Cpu": "2 vCPU",
        "Memory": "4 GB"
    }
}

In [20]:
with open('app_runner_config.json', 'w') as f:
    json.dump(conf, f)

In [21]:
!aws apprunner create-service --cli-input-json file://app_runner_config.json


An error occurred (AccessDeniedException) when calling the CreateService operation: Account 654654589924 is not authorized pass this role for operation CreateService


# Testing the Model API

In [72]:
!curl -X POST "<Default domain>/predict/" \
  -H "Content-Type: application/json" \
  -d '{"features": [1,2,3,4,5]}'

{"prediction":0.0018256510375067592}

In [ ]:
# 1)
# Try it out in https://reqbin.com/ 
# with the body:
# {"features": [1,2,3,4,5]}

In [ ]:
# 2)
# Download the file "card_fraud_detection_portal.html"
# replace the API domain in the file and open the file in your browser 

# Delete Image

In [ ]:
!docker rmi <IMAGE_ID>   
!docker rmi <FULL_TAG_NAME>

In [162]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE
